<a href="https://colab.research.google.com/github/Sang897/hello-world/blob/master/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Data Science HW 9
##### Copyright Jeffrey Stanton, Jeffrey Saltz, and Jasmina Tacheva

In [ ]:
# Enter your name here: Jack (Sang Uk) Park 

### Attribution statement: (choose only one and delete the rest)

In [ ]:
# 1. I did this homework by myself, with help from the book and the professor.
# 2. I did this homework with help from the book and the professor and these Internet sources:
# 3. I did this homework with help from <Name of another student> but did not cut and paste any code.

Supervised learning means that there is a **criterion one is trying to predict**. The typical strategy is to **divide data** into a **training set** and a **test set** (for example, **two-thirds training** and **one-third test**), train the model on the training set, and then see how well the model does on the test set. <br>

**Support vector machines (SVM)** are a highly flexible and powerful method of doing **supervised machine learning**.

Another approach is to use **partition trees (rpart)** 

In this homework, we will use another banking dataset to train an SVM model, as well as an rpart model, to **classify potential borrowers into 2 groups of credit risk** – **reliable borrowers** and **borrowers posing a risk**. You can learn more about the variables in the dataset here:<br> https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29 <br>

This kind of classification algorithms is used in many aspects of our lives – from credit card approvals to stock market predictions, and even some medical diagnoses. <br>

## Part 1: Load and condition the data  

A.	Read the contents of the following .csv file into a dataframe called **credit**: <br>

https://intro-datascience.s3.us-east-2.amazonaws.com/GermanCredit.csv <br>

You will also need to install( ) and library( ) several other libraries, such as **kernlab** and **caret**.


In [6]:
install.packages("rpart.plot")
install.packages("caret")
install.packages("rpart")

install.packages("kernlab")
install.packages("caret")
library(rpart)
library(rpart.plot)
library(caret)
library(Rcpp)
library(readr)
library(ggplot2)

credit <-read_csv("https://intro-datascience.s3.us-east-2.amazonaws.com/GermanCredit.csv")
credit

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Rows: 1000 Columns: 21

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): status, credit_history, purpose, savings, employment_duration, per...
dbl  (8): duration, amount, installment_rate, present_residence, age, number...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,⋯,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
... < 100 DM,6,critical account/other credits existing,domestic appliances,1169,unknown/no savings account,... >= 7 years,4,male : single,none,⋯,real estate,67,none,own,2,skilled employee/official,1,yes,yes,1
0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,none,⋯,real estate,22,none,own,1,skilled employee/official,1,no,yes,0
no checking account,12,critical account/other credits existing,retraining,2096,... < 100 DM,4 <= ... < 7 years,2,male : single,none,⋯,real estate,49,none,own,1,unskilled - resident,2,no,yes,1
... < 100 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,guarantor,⋯,building society savings agreement/life insurance,45,none,for free,1,skilled employee/official,2,no,yes,1
... < 100 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,none,⋯,unknown/no property,53,none,for free,2,skilled employee/official,2,no,yes,0
no checking account,36,existing credits paid back duly till now,retraining,9055,unknown/no savings account,1 <= ... < 4 years,2,male : single,none,⋯,unknown/no property,35,none,for free,1,unskilled - resident,2,yes,yes,1
no checking account,24,existing credits paid back duly till now,radio/television,2835,500 <= ... < 1000 DM,... >= 7 years,3,male : single,none,⋯,building society savings agreement/life insurance,53,none,own,1,skilled employee/official,1,no,yes,1
0 <= ... < 200 DM,36,existing credits paid back duly till now,car (used),6948,... < 100 DM,1 <= ... < 4 years,2,male : single,none,⋯,car or other,35,none,rent,1,management/self-employed/highly qualified employee/officer,1,yes,yes,1
no checking account,12,existing credits paid back duly till now,domestic appliances,3059,... >= 1000 DM,4 <= ... < 7 years,2,male : divorced/separated,none,⋯,real estate,61,none,own,1,unskilled - resident,1,no,yes,1


B.	Which variable contains the outcome we are trying to predict, **credit risk**? For the purposes of this analysis, we will focus only on the numeric variables and save them in a new dataframe called **cred**:

In [7]:
cred <- data.frame(duration=credit$duration, 
                   amount=credit$amount, 
                   installment_rate=credit$installment_rate, 
                   present_residence=credit$present_residence, 
                   age=credit$age, 
                   credit_history=credit$number_credits, 
                   people_liable=credit$people_liable, 
                   credit_risk=as.factor(credit$credit_risk))

C.	Although all variables in **cred** except **credit_risk** are coded as numeric, the values of one of them are also **ordered factors** rather than actual numbers. In consultation with the **data description link** from the intro, write a comment identifying the **factor variable** and briefly **describe** each variable in the dataframe. 

In [8]:

is.factor(cred$credit_risk)
sapply(cred, class)

[1] TRUE

duration            amount  installment_rate present_residence 
        "numeric"         "numeric"         "numeric"         "numeric" 
              age    credit_history     people_liable       credit_risk 
        "numeric"         "numeric"         "numeric"          "factor"

## Part 2: Create training and test data sets

A.	Using techniques discussed in class, create **two datasets** – one for **training** and one for **testing**.

In [10]:
trainList <-createDataPartition(y=cred$credit_risk,p=.40,list=FALSE) 
trainSet <- cred[trainList,]
testSet <- cred[-trainList,]

B.	Use the dim( ) function to demonstrate that the resulting training data set and test data set contain the appropriate number of cases.

In [11]:
dim(trainSet)
dim(testSet)
head(trainSet)

[1] 400   8

[1] 600   8

,duration,amount,installment_rate,present_residence,age,credit_history,people_liable,credit_risk
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
3,12,2096,2,3,49,1,2,1
4,42,7882,2,4,45,1,2,1
5,24,4870,3,4,53,2,2,0
9,12,3059,2,4,61,1,1,1
10,30,5234,4,2,28,2,1,0
11,12,1295,3,1,25,1,1,0


## Part 3: Build a Model using SVM

A.	Using the caret package, build a support vector model using all of the variables to predict **credit_risk**

In [13]:
SV <- train(credit_risk ~ ., data=trainSet, method="svmRadial", preProc=c("center","scale"))

B. output the model

Hint: explore finalModel in the model that would created in F.

In [14]:
SV

Support Vector Machines with Radial Basis Function Kernel 

400 samples
  7 predictor
  2 classes: '0', '1' 

Pre-processing: centered (7), scaled (7) 
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 400, 400, 400, 400, 400, 400, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa     
  0.25  0.7121101  0.05649635
  0.50  0.7125150  0.11510750
  1.00  0.7093325  0.14789220

Tuning parameter 'sigma' was held constant at a value of 0.1197401
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.1197401 and C = 0.5.

## Part 4: Predict Values in the Test Data and Create a Confusion Matrix

A.	Use the predict( ) function to validate the model against test data. Store the predictions in a variable named **svmPred**.

In [15]:
svmPred <- predict(SV, newdata=testSet)

In [16]:
svmPred

[1] 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 [75] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[112] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[149] 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[186] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[223] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
[260] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[297] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[334] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[371] 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[408] 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
[445] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
[482] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[519] 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[556] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[593] 1 1 1 1 1 1 1 0
Levels: 0 1

In [17]:
testSet

,duration,amount,installment_rate,present_residence,age,credit_history,people_liable,credit_risk
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,1169,4,4,67,2,1,1
2,48,5951,2,2,22,1,1,0
6,36,9055,2,4,35,1,2,1
7,24,2835,3,4,53,1,1,1
8,36,6948,2,2,35,1,1,1
12,48,4308,3,4,24,1,1,0
13,12,1567,1,1,22,1,1,1
14,24,1199,4,4,60,2,1,0
16,24,1282,4,2,32,1,1,0


In [ ]:
predictValues

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[112] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[149] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[186] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[223] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[260] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[297] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[334] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[371] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[408] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[445] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[482] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[519] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[556] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[593] 1 1 1 1 1 1 1 1
Levels: 0 1

B.	The **svmPred** object contains a list of classifications for reliable (=0) or risky (=1) borrowers. Review the contents of **svmPred** using head( ).

In [18]:
head(svmPred)

#So the SVMPred object contains either 0 or 1 based on the testset that was created. 

[1] 1 1 1 1 1 0
Levels: 0 1

C.	Explore the **confusion matrix**, using the caret package

In [23]:
library(caret)
confusion <- confusionMatrix(svmPred, testSet$credit_risk)
confusion 

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0  11   9
         1 169 411
                                         
               Accuracy : 0.7033         
                 95% CI : (0.665, 0.7396)
    No Information Rate : 0.7            
    P-Value [Acc > NIR] : 0.4492         
                                         
                  Kappa : 0.0532         
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 0.06111        
            Specificity : 0.97857        
         Pos Pred Value : 0.55000        
         Neg Pred Value : 0.70862        
             Prevalence : 0.30000        
         Detection Rate : 0.01833        
   Detection Prevalence : 0.03333        
      Balanced Accuracy : 0.51984        
                                         
       'Positive' Class : 0              
                                         

D.	What is the **accuracy** based on what you see in the confusion matrix. 

In [ ]:
accuracy is 0.5 or 50 percent. 

E.	Compare your calculations with the **confusionMatrix()** function from the **caret** package.

In [2]:
install.packages("caret")
library(caret)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


F.	Explain, in a block comment:<br> 1) why it is valuable to have a “test” dataset that is separate from a “training” dataset, and <br>2) what potential ethical challenges this type of automated classification may pose. 

In [ ]:
#The reason for having a test is because we don't want to use the entire set to test the data. We always want to have around 80 percent as the
#training set and leave 20 percent of the rest, so that we can retest and make sure that the training set's result is also applied in the test set.
#So the test set helps to validate the model as the test set itself is unbiased and not corrected from the train set. 

## Part 5: Now build a tree model (with rpart)

A. Build a model with rpart
<br>
Note: you might need to install the e1071 package

In [25]:
install.packages("e1071")
library(e1071)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [40]:
cartTree1 <- train(credit_risk~ ., data=trainSet, method="rpart")
cartTree1

CART 

400 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 400, 400, 400, 400, 400, 400, ... 
Resampling results across tuning parameters:

  cp          Accuracy   Kappa    
  0.01250000  0.6621131  0.1428174
  0.03333333  0.6903913  0.1396918
  0.05833333  0.6971794  0.1196958

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was cp = 0.05833333.

B. Visualize the results using  rpart.plot()

In [39]:
 rpart.plot(cartTree1)

 cartTree1

ERROR: ignored

C. Use the **predict()** function to predict the testData, and then generate a confusion matrix to explore the results

In [34]:
predOut <- predict(cartTree1, newdata=testSet)
confusion <- confusionMatrix(predOut, testSet$credit_risk)


D. Review the attributes being used for this credit decision. Are there any that might not be appropriate, with respect to fairness? If so, which attribute, and how would you address this fairness situation. Answer in a comment block below

In [ ]:
#I think this situation can be more fair if person's income level is included. I think a person's income level is related to the credit risk level. 

In [32]:
#Yes. There is no gender in the factor. So 